In [2]:
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# defining the server and the database
server = '(localdb)\mssqllocaldb' 
database = 'AdventureWorks2012'  

# Define the connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE ='+ database +';\
    Trusted_Connection=yes;'
) 

###  1. Get the name and list price for products which have a list price greater than the average product


In [12]:
# Thought Process:
# Using the subquery filter ListPrice of products greater than AVG() ListPrice of all products
# Using table [AdventureWorks2012].[Production].[Product] AS Prod
# Using the products list price > the average list price in the WHERE clause
# Outer query using SELECT statment to get Product ID, Product Name and List Price

query = pd.read_sql_query("""SELECT Prod.ProductID AS [ID], 
                             Prod.Name AS [Product Name], 
                             FORMAT(Prod.ListPrice,'C') AS [Price]
                             FROM [AdventureWorks2012].[Production].[Product] AS Prod
                             WHERE Prod.ListPrice > (SELECT AVG(Prod.ListPrice)
                                                      FROM[AdventureWorks2012].[Production].[Product] AS Prod)
                             ORDER BY Prod.ProductID""",conn)
query

,ID,Product Name,Price
0,680,"HL Road Frame - Black, 58","$1,431.50"
1,706,"HL Road Frame - Red, 58","$1,431.50"
2,717,"HL Road Frame - Red, 62","$1,431.50"
3,718,"HL Road Frame - Red, 44","$1,431.50"
4,719,"HL Road Frame - Red, 48","$1,431.50"
...,...,...,...
131,992,"Mountain-500 Black, 48",$539.99
132,993,"Mountain-500 Black, 52",$539.99
133,997,"Road-750 Black, 44",$539.99
134,998,"Road-750 Black, 48",$539.99


### 2. Get the name and list price for products then add to the query a 
### subquery (in the select part) to get the product model (field “name” in ProductModel table). 

In [15]:
# Outer SELECT get Production Name and ListPrice field
# Then add a nested SELECT in query to get name field 
# FROM [AdventureWorks2012].[Production].[ProductModel] AS Mod table
# Use FROM for the [AdventureWorks2012].[Production].[Product] AS Prod table

query = pd.read_sql_query("""SELECT Prod.Name,

                             (SELECT Mod.Name 
                              FROM [AdventureWorks2012].[Production].[ProductModel] AS Mod
                              WHERE Mod.ProductModelID = Prod.ProductModelID),
                              
                             Prod.ListPrice
                             FROM [AdventureWorks2012].[Production].[Product] AS Prod""",conn)
query

,Name,,ListPrice
0,Adjustable Race,None,0.00
1,Bearing Ball,None,0.00
2,BB Ball Bearing,None,0.00
3,Headset Ball Bearings,None,0.00
4,Blade,None,0.00
...,...,...,...
499,ML Bottom Bracket,ML Bottom Bracket,101.24
500,HL Bottom Bracket,HL Bottom Bracket,121.49
501,"Road-750 Black, 44",Road-750,539.99
502,"Road-750 Black, 48",Road-750,539.99


### 3. Add to query in 2) a subquery (in the where clause) to filter on 
### products for which more than 5 have been ordered in one order 
### (Hint: OrderQty field in SalesOrderDetail)

In [4]:
# Thought Process:
# Use tables [AdventureWorks2012].[Production].[ProductModel] AS Mod
# table [AdventureWorks2012].[Production].[Product] AS Prod
# table [AdventureWorks2012].[Sales].[SalesOrderDetail] AS SOD
# IN subquery filter records WHERE OrderQty > 5
# Nested SELECT query for model name field in Product Model table
# In the Outer query get Product name field and list price field

query = pd.read_sql_query("""SELECT Prod.Name,

                             (SELECT Mod.Name 
                              FROM [AdventureWorks2012].[Production].[ProductModel] AS Mod
                              WHERE Mod.ProductModelID = Prod.ProductModelID),
                              
                             FORMAT(Prod.ListPrice,'C')
                             FROM [AdventureWorks2012].[Production].[Product] AS Prod
                             
                             WHERE Prod.ProductID IN (SELECT SOD.ProductID
                                                      FROM [AdventureWorks2012].[Sales].[SalesOrderDetail] AS SOD
                                                      WHERE SOD.OrderQty > 5)
                             
                              ORDER BY Prod.ListPrice""",conn)
query

,Name,,
0,Patch Kit/8 Patches,Patch kit,$2.29
1,Water Bottle - 30 oz.,Water Bottle,$4.99
2,Bike Wash - Dissolver,Bike Wash,$7.95
3,"Racing Socks, M",Racing Socks,$8.99
4,"Racing Socks, L",Racing Socks,$8.99
...,...,...,...
222,"Mountain-100 Silver, 48",Mountain-100,"$3,399.99"
223,"Road-150 Red, 44",Road-150,"$3,578.27"
224,"Road-150 Red, 48",Road-150,"$3,578.27"
225,"Road-150 Red, 52",Road-150,"$3,578.27"


### 4. Can you achieve the same results as 3) by writing a query without sub queries in it?




In [30]:
# I will use INNER JOIN to get same results
# Aim is to join three table and get fields with using on SELECT query
# Using Left Outer Join for [AdventureWorks2012].[Production].[Product] AS Prod
# AND [AdventureWorks2012].[Production].[ProductModel] AS Mod
# Then use INNER JOIN [AdventureWorks2012].[Sales].[SalesOrderDetail] AS SOD
# Keep WHERE clause SOD.OrderQty > 5 to filter results
# First Attempt getting more than 10000 records
# Fix implement DISTINCT keyword to reduce reduncancy
# Result matches results as 3.

query = pd.read_sql_query("""SELECT DISTINCT Prod.Name AS [Product],
                             Mod.Name AS [Model], 
                             FORMAT(Prod.ListPrice,'C')
                             
                             FROM [AdventureWorks2012].[Production].[Product] AS Prod
                             LEFT OUTER JOIN [AdventureWorks2012].[Production].[ProductModel] AS Mod
                             ON Mod.ProductModelID = Prod.ProductModelID
                             INNER JOIN [AdventureWorks2012].[Sales].[SalesOrderDetail] AS SOD
                             ON SOD.ProductID = Prod.ProductID
                             
                             WHERE SOD.OrderQty > 5""",conn)
query


,Product,Model,
0,ML Headset,ML Headset,$102.29
1,"Racing Socks, M",Racing Socks,$8.99
2,"Mountain-300 Black, 38",Mountain-300,"$1,079.99"
3,"Full-Finger Gloves, M",Full-Finger Gloves,$37.99
4,"Women's Mountain Shorts, M",Women's Mountain Shorts,$69.99
...,...,...,...
222,Cable Lock,Cable Lock,$25.00
223,"Touring-2000 Blue, 54",Touring-2000,"$1,214.85"
224,"Road-650 Black, 62",Road-650,$782.99
225,"LL Touring Frame - Yellow, 44",LL Touring Frame,$333.42


### 5. To test your code: Combine both 3) and 4) via an EXCEPT. The result of 5) should yield no records.



In [17]:
query = pd.read_sql_query("""SELECT Prod.Name AS [Product Name],

                             (SELECT Mod.Name 
                              FROM [AdventureWorks2012].[Production].[ProductModel] AS Mod
                              WHERE Mod.ProductModelID = Prod.ProductModelID) AS [Product Model],
                              
                             Prod.ListPrice AS [Price $]
                             FROM [AdventureWorks2012].[Production].[Product] AS Prod
                             
                             WHERE Prod.ProductID IN (SELECT SOD.ProductID
                                                      FROM [AdventureWorks2012].[Sales].[SalesOrderDetail] AS SOD
                                                      WHERE SOD.OrderQty > 5)                             
                             EXCEPT
                             
                             SELECT Prod.Name AS [Product Name],
                             Mod.Name AS [Product Model], 
                             Prod.ListPrice AS [Price $]
                             
                             FROM [AdventureWorks2012].[Production].[Product] AS Prod
                             LEFT OUTER JOIN [AdventureWorks2012].[Production].[ProductModel] AS Mod
                             ON Mod.ProductModelID = Prod.ProductModelID
                             INNER JOIN [AdventureWorks2012].[Sales].[SalesOrderDetail] AS SOD
                             ON SOD.ProductID = Prod.ProductID
                             
                             WHERE SOD.OrderQty > 5""",conn)
query

,Product Name,Product Model,Price $
